# ネットワークモデルのTorchScript化(JIT化)
### 特徴
+ PythonのGILを回避できる
+ 他のプラットフォームと相互運用性が必要ない場合に使用する(必要な場合はONNX)
+ ONNXファイルもJIT化を使用して実現している

### JIT化の方法
| Method | 特徴 |
| :-- | :-- |
| trace | JIT化したいモデルに入力と同じ形状のダミー入力を与えて, トレース中に辿ったコードのみをJIT化する |
| script | JIT化したいモデルを全てJIT化しようとするが, 専用のコンパイラが理解できる範囲という制限がある |

### Traceする際は, モデルパラメータの勾配計算を全てOFFにする
```
for p in seg_model.parameters():
  p.requires_grad_(False)

dummy_input = torch.randn(1, 8, 512, 512)
traced_seg_model = torch.jit.trace(seg_model, dummy_input) # JIT化(TorchScript)
torch.jit.save(traced_seg_model, 'traced_seg_model.pt')
loaded_seg_model = torch.jit.load('traced_seg_model.pt')
```

In [1]:
import torch
import re

In [2]:
torch.__file__

'c:\\Users\\inoue\\anaconda3\\envs\\Py39WorkingEnv\\lib\\site-packages\\torch\\__init__.py'

In [2]:
def xprint(s):
    s = str(s)
    s = re.sub(' *#.*','',s)
    print(s)

In [4]:
# JIT化対象の関数(例)
def myfn(x):
    y = x[0]
    for i in range(1, x.size(0)):
        y = y + x[i] # x[0]~x[x.size(0)-1]までを加算
    return y

トレース

In [5]:
inp = torch.randn(5,5)

with torch.no_grad():
    traced_fn = torch.jit.trace(myfn, inp)

print(traced_fn.code)

def myfn(x: Tensor) -> Tensor:
  y = torch.select(x, 0, 0)
  y0 = torch.add(y, torch.select(x, 0, 1))
  y1 = torch.add(y0, torch.select(x, 0, 2))
  y2 = torch.add(y1, torch.select(x, 0, 3))
  return torch.add(y2, torch.select(x, 0, 4))



スクリプト

In [8]:
scripted_fn = torch.jit.script(myfn)
print(scripted_fn.code)

def myfn(x: Tensor) -> Tensor:
  y = torch.select(x, 0, 0)
  _0 = torch.__range_length(1, torch.size(x, 0), 1)
  y0 = y
  for _1 in range(_0):
    i = torch.__derive_index(_1, 1, 1)
    y1 = torch.add(y0, torch.select(x, 0, i))
    y0 = y1
  return y0



スクリプトのグラフ表現

In [9]:
print(scripted_fn.graph)

graph(%x.1 : Tensor):
  %10 : bool = prim::Constant[value=1]() # C:\Users\inoue\AppData\Local\Temp\ipykernel_17040\3746942711.py:4:4
  %2 : int = prim::Constant[value=0]() # C:\Users\inoue\AppData\Local\Temp\ipykernel_17040\3746942711.py:3:10
  %5 : int = prim::Constant[value=1]() # C:\Users\inoue\AppData\Local\Temp\ipykernel_17040\3746942711.py:4:19
  %y.1 : Tensor = aten::select(%x.1, %2, %2) # C:\Users\inoue\AppData\Local\Temp\ipykernel_17040\3746942711.py:3:8
  %7 : int = aten::size(%x.1, %2) # C:\Users\inoue\AppData\Local\Temp\ipykernel_17040\3746942711.py:4:22
  %9 : int = aten::__range_length(%5, %7, %5) # C:\Users\inoue\AppData\Local\Temp\ipykernel_17040\3746942711.py:4:4
  %y : Tensor = prim::Loop(%9, %10, %y.1) # C:\Users\inoue\AppData\Local\Temp\ipykernel_17040\3746942711.py:4:4
    block0(%11 : int, %y.11 : Tensor):
      %i.1 : int = aten::__derive_index(%11, %5, %5) # C:\Users\inoue\AppData\Local\Temp\ipykernel_17040\3746942711.py:4:4
      %19 : Tensor = aten::select(%x.